In [4]:
# =====================================================
#  Clean Slate (Run this FIRST)
# =====================================================
%reset -f

# %% [markdown]
# # Match_NCES_to_BCS
# 
# This project merges the **New York State Building Condition Survey (BCS)** data
# with the **National Center for Education Statistics (NCES)** public and private school datasets.
#
# **Project folder:** `Match_NCES_to_BCS`
# 
# **Outputs:** All results are saved under `/output/`
#
# **Author:** Elif Yegenoglu  
# **Last Updated:** October 2025

# %% [code]
# =====================================================
# STEP 0 — Ensure correct working directory
# =====================================================

import os
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook

cwd = Path.cwd().resolve()
if cwd.name == "notebooks" or (cwd / "notebooks").exists():
    os.chdir("..")
    print(f"Changed working directory to project root: {Path.cwd().resolve()}")
else:
    print(f"Running from: {cwd}")

# =====================================================
# STEP 1 — Directory Setup
# =====================================================
import pandas as pd
from openpyxl import load_workbook

BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR / "data"
OUTPUT_DIR = BASE_DIR / "output"
BCS_DIR = DATA_DIR / "BCS"
NCES_DIR = DATA_DIR / "NCES"
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Project root: {BASE_DIR.name}")
print(f"Data folder: {DATA_DIR}")
print(f"Output folder: {OUTPUT_DIR}")

# =====================================================
# STEP 2 — Merge Four BCS Excel Files
# =====================================================
bcs_files = sorted(BCS_DIR.glob("NY_BCS_BuildingAge_*.xlsx"))

if not bcs_files:
    print("⚠️ No BCS files found. Check the folder:", BCS_DIR)
    print("Files present:", [f.name for f in BCS_DIR.glob('*')])
    raise FileNotFoundError("No BCS Excel files found in the expected directory.")

print(f"Loading {len(bcs_files)} BCS files...")
bcs_dfs = [pd.read_excel(f, engine="openpyxl") for f in bcs_files]
bcs_merged = pd.concat(bcs_dfs, ignore_index=True)
print(f"Combined BCS dataset shape: {bcs_merged.shape}")

# Save merged BCS
bcs_merged.to_excel(OUTPUT_DIR / "BCS_Merged_AllYears.xlsx", index=False)
print("Saved merged BCS file → output/BCS_Merged_AllYears.xlsx")

# Clean ZIPs
bcs_cols = [
    "School_District", "BEDS_Code", "Building_911_Address", "City",
    "Zip_Code", "Original Construction",
    "Addition #1", "Addition #2", "Addition #3", "Addition #4",
    "Addition #5", "Addition #6", "Addition #7", "Addition #8", "Addition #9"
]
bcs = bcs_merged[bcs_cols].copy()
bcs["Zip_Code"] = bcs["Zip_Code"].astype(str).str.extract(r"(\d{5})")[0]
print(f"Unique ZIPs in BCS: {bcs['Zip_Code'].nunique()}")

# %% [markdown]
# ## Step 3 — Load NCES Excel Files with Header Detection

# %% [code]
def detect_header_row_xlsx(file_path):
    """Detect which row contains the header by searching for 'ZIP' or 'School Name'."""
    wb = load_workbook(file_path, read_only=True)
    ws = wb.active
    for i, row in enumerate(ws.iter_rows(values_only=True), start=0):
        if not row or all(v is None for v in row):
            continue
        row_values = [str(v).strip().lower() for v in row if v]
        if any("zip" in v or "school name" in v for v in row_values):
            wb.close()
            print(f"Detected header row {i} in '{file_path}'")
            return i
    wb.close()
    raise ValueError(f"Could not detect header row in {file_path}")

def read_nces_excel_auto(file_path):
    """Reads Excel using detected header row and cleans column names."""
    header_row = detect_header_row_xlsx(file_path)
    df = pd.read_excel(file_path, skiprows=header_row, engine="openpyxl")
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace("\xa0", "", regex=True)
        .str.lower()
    )
    print(f"Loaded {file_path.name} (header at row {header_row}): {df.shape}")
    display(df.head(3))
    return df

# Verify files exist
for required in ["NCESdata_Public.xlsx", "NCESdata_Private.xlsx"]:
    if not (NCES_DIR / required).exists():
        raise FileNotFoundError(f"Missing {required} in /data/NCES/")

nces_public = read_nces_excel_auto(NCES_DIR / "NCESdata_Public.xlsx")
nces_private = read_nces_excel_auto(NCES_DIR / "NCESdata_Private.xlsx")

# %% [markdown]
# ## Step 4 — Standardize and Combine NCES Datasets

# %% [code]
rename_map_private = {
    "pss_school_id": "nces_school_id",
    "pss_inst": "school_name",
    "pss_address": "street_address",
    "pss_city": "city",
    "pss_stabb": "state",
    "pss_zip5": "zip",
    "pss_county_name": "county_name",
    "pss_fte_teach": "teachers",
    "pss_enroll_t": "students",
    "pss_level": "type",
    "pss_relig": "charter"
}
nces_private = nces_private.rename(columns=rename_map_private)

# Align schemas
for col in ["nces_school_id", "school_name", "city", "state", "zip", "county_name", "students", "teachers"]:
    if col not in nces_public.columns:
        nces_public[col] = None
    if col not in nces_private.columns:
        nces_private[col] = None

nces_public["Source"] = "Public"
nces_private["Source"] = "Private"

# Clean ZIPs
zip_public = [c for c in nces_public.columns if "zip" in c][0]
zip_private = [c for c in nces_private.columns if "zip" in c][0]
nces_public["zip_clean"] = nces_public[zip_public].astype(str).str.extract(r"(\d{5})")[0]
nces_private["zip_clean"] = nces_private[zip_private].astype(str).str.extract(r"(\d{5})")[0]

# Merge both NCES datasets
nces_merged = pd.concat([nces_public, nces_private], ignore_index=True, sort=False)

# Preserve IDs as text
for id_col in ["nces_school_id", "nces_district_id"]:
    if id_col in nces_merged.columns:
        nces_merged[id_col] = (
            nces_merged[id_col]
            .astype(str)
            .str.replace(r"\.0$", "", regex=True)
            .str.strip()
            .apply(lambda x: x if x.lower() not in ["nan", "none"] else None)
        )

print(f"Merged NCES dataset shape: {nces_merged.shape}")
display(nces_merged.head(3))

# %% [markdown]
# ## Step 5 — ZIP Coverage Diagnostics

# %% [code]
public_zips = set(nces_public["zip_clean"].dropna())
private_zips = set(nces_private["zip_clean"].dropna())
only_in_public = sorted(public_zips - private_zips)
only_in_private = sorted(private_zips - public_zips)

print(f"Public rows: {len(nces_public)}")
print(f"Private rows: {len(nces_private)}")
print(f"Shared ZIPs: {len(public_zips & private_zips)}")
print(f"ZIPs only in Public: {len(only_in_public)}")
print(f"ZIPs only in Private: {len(only_in_private)}")

pd.DataFrame({"ZIP_Only_in_Public": only_in_public}).to_excel(OUTPUT_DIR / "ZIPs_Only_in_Public.xlsx", index=False)
pd.DataFrame({"ZIP_Only_in_Private": only_in_private}).to_excel(OUTPUT_DIR / "ZIPs_Only_in_Private.xlsx", index=False)

# %% [markdown]
# ## Step 6 — Prepare BCS Data for ZIP Merge

# %% [code]
bcs_sorted = bcs.sort_values(by=["Zip_Code", "School_District"], na_position="last")
bcs_unique = bcs_sorted.drop_duplicates(subset="Zip_Code", keep="first")
print(f"BCS reduced from {len(bcs_sorted)} to {len(bcs_unique)} unique ZIP rows")

# %% [markdown]
# ## Step 7 — Merge NCES and BCS by ZIP Code

# %% [code]
nces_final = pd.merge(
    nces_merged,
    bcs_unique,
    how="left",
    left_on="zip_clean",
    right_on="Zip_Code"
)
print(f"Final merged dataset shape: {nces_final.shape}")
display(nces_final.head(3))

# %% [markdown]
# ## Step 8 — Match Diagnostics by Source and Erie County Check

# %% [code]
match_summary = (
    nces_final
    .assign(BCS_Matched=nces_final["School_District"].notna())
    .groupby("Source")["BCS_Matched"]
    .value_counts()
    .unstack(fill_value=0)
)
match_summary["Match_%"] = (
    100 * match_summary[True] / (match_summary[True] + match_summary[False])
).round(1)

# Erie County Match Check
if "county_name" in nces_final.columns:
    erie_all = nces_final[nces_final["county_name"].str.contains("Erie", case=False, na=False)]
    erie_matched = erie_all[erie_all["School_District"].notna()]
    erie_total = len(erie_all)
    erie_matched_count = len(erie_matched)
    erie_match_pct = round(100 * erie_matched_count / erie_total, 1) if erie_total > 0 else 0

    print(f"\nErie County: {erie_matched_count} matched out of {erie_total} total ({erie_match_pct}%)")
else:
    print("\n⚠️ No 'county_name' column found to check Erie County matches.")

display(match_summary)
match_summary.to_excel(OUTPUT_DIR / "BCS_Match_Summary_by_Source.xlsx")

# %% [markdown]
# ## Step 9 — Final Cleanup and Save

# %% [code]
blank_cols = [c for c in nces_final.columns if nces_final[c].isna().mean() > 0.95]
non_blank_cols = [c for c in nces_final.columns if c not in blank_cols]
nces_final = nces_final[non_blank_cols + blank_cols]
print(f"Reordered columns: {len(non_blank_cols)} with data, {len(blank_cols)} mostly blank moved to end")

# Save all key outputs
nces_merged.to_excel(OUTPUT_DIR / "NCES_PublicPrivate_Merged.xlsx", index=False)
nces_final.to_excel(OUTPUT_DIR / "NCES_to_BCS_Link.xlsx", index=False)  # renamed here
match_summary.to_excel(OUTPUT_DIR / "BCS_Match_Summary_by_Source.xlsx", index=True)

print("\nAll outputs saved in the /output folder.")

# %% [code]
# =====================================================
# STEP 10 — Environment Info
# =====================================================
import sys, platform
print("\n--- Environment Info ---")
print(f"Python version: {sys.version.split()[0]}")
print(f"Pandas version: {pd.__version__}")
print(f"Platform: {platform.system()} {platform.release()}")
print("Processing complete.")


Running from: C:\Users\yegen\Downloads\Match_NCES_to_BCS
Project root: Match_NCES_to_BCS
Data folder: C:\Users\yegen\Downloads\Match_NCES_to_BCS\data
Output folder: C:\Users\yegen\Downloads\Match_NCES_to_BCS\output
Loading 4 BCS files...
Combined BCS dataset shape: (3471, 20)
Saved merged BCS file → output/BCS_Merged_AllYears.xlsx
Unique ZIPs in BCS: 787
Detected header row 11 in 'C:\Users\yegen\Downloads\Match_NCES_to_BCS\data\NCES\NCESdata_Public.xlsx'
Loaded NCESdata_Public.xlsx (header at row 11): (4816, 26)


,nces_school_id,state_school_id,nces_district_id,state_district_id,low_grade,high_grade,school_name,district,county_name,street_address,...,locale,charter,students,teachers,student_teacher_ratio,free_lunch,reduced_lunch,directly_certified,type,status
0,360010206477,NY-343000010000-343000010300,3600102,NY-343000010000,KG,08,30TH AVENUE SCHOOL (THE) (G & T CITYWIDE),NEW YORK CITY GEOGRAPHIC DISTRICT #30,Queens County,28-37 29TH ST,...,"City, Large",No,513.00000,26.71000,19.2100000,130.00000,12.00000,–,Regular,Open
1,360007705767,NY-310200010000-310200010347,3600077,NY-310200010000,PK,08,47 AMERICAN SIGN LANGUAGE AND ENGLISH LOWER SC...,NEW YORK CITY GEOGRAPHIC DISTRICT # 2,New York County,223 E 23RD ST,...,"City, Large",No,177.00000,30.10000,5.8800000,137.00000,0.00000,–,Regular,Open
2,362343003270,NY-051101040000-051101040001,3623430,NY-051101040000,PK,06,A A GATES ELEMENTARY SCHOOL,PORT BYRON CENTRAL SCHOOL DISTRICT,Cayuga County,30 MAPLE AVE,...,"Rural, Distant",No,440.00000,33.05000,13.3100000,191.00000,0.00000,–,Regular,Open


Detected header row 4 in 'C:\Users\yegen\Downloads\Match_NCES_to_BCS\data\NCES\NCESdata_Private.xlsx'
Loaded NCESdata_Private.xlsx (header at row 4): (1229, 71)


,pss_school_id,pss_inst,lograde,higrade,pss_address,pss_city,pss_county_no,pss_county_fips,pss_stabb,pss_fips,...,pss_assoc_6,pss_assoc_7,pss_assoc_8,pss_assoc_9,pss_assoc_10,pss_assoc_11,pss_assoc_12,pss_assoc_13,pss_assoc_14,pss_assoc_15
0,BB061076,A Fantis School,2,13,195 STATE ST,BROOKLYN,36047,47,NY,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1902950,A PLUS KIDZ ACADEMY,2,6,10510 FLATLANDS AVE,BROOKLYN,36047,47,NY,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A1902951,AARON SCHOOL K - 12,13,17,42 E 30TH ST,NEW YORK,36061,61,NY,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Merged NCES dataset shape: (6045, 88)


...

Public rows: 4816
Private rows: 1229
Shared ZIPs: 446
ZIPs only in Public: 622
ZIPs only in Private: 51
BCS reduced from 3471 to 788 unique ZIP rows
Final merged dataset shape: (6045, 103)


,nces_school_id,state_school_id,nces_district_id,state_district_id,low_grade,high_grade,school_name,district,county_name,street_address,...,Original Construction,Addition #1,Addition #2,Addition #3,Addition #4,Addition #5,Addition #6,Addition #7,Addition #8,Addition #9
0,360010206477,NY-343000010000-343000010300,3600102,NY-343000010000,KG,08,30TH AVENUE SCHOOL (THE) (G & T CITYWIDE),NEW YORK CITY GEOGRAPHIC DISTRICT #30,Queens County,28-37 29TH ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,360007705767,NY-310200010000-310200010347,3600077,NY-310200010000,PK,08,47 AMERICAN SIGN LANGUAGE AND ENGLISH LOWER SC...,NEW YORK CITY GEOGRAPHIC DISTRICT # 2,New York County,223 E 23RD ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,362343003270,NY-051101040000-051101040001,3623430,NY-051101040000,PK,06,A A GATES ELEMENTARY SCHOOL,PORT BYRON CENTRAL SCHOOL DISTRICT,Cayuga County,30 MAPLE AVE,...,1990,2017,(No Response),(No Response),(No Response),(No Response),(No Response),(No Response),(No Response),(No Response)



Erie County: 263 matched out of 275 total (95.6%)


BCS_Matched,False,True,Match_%
Source,,,
Private,679,550,44.8
Public,2273,2543,52.8


Reordered columns: 88 with data, 15 mostly blank moved to end

All outputs saved in the /output folder.

--- Environment Info ---
Python version: 3.12.11
Pandas version: 2.3.3
Platform: Windows 11
Processing complete.
